# Preliminaries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Set working directory
from google.colab import drive, runtime
drive.mount('/content/gdrive')
%cd gdrive/My Drive/MLHC/satellite_semantic_segmentation/src

Mounted at /content/gdrive
/content/gdrive/My Drive/MLHC/satellite_semantic_segmentation/src


In [3]:
# Install relevant packages
%pip install segment-geospatial groundingdino-py localtileserver
from samgeo import tms_to_geotiff
from samgeo.text_sam import LangSAM
from individual_model import train_individual_model
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.3/263.3 kB 33.4 MB/s eta 0

In [4]:
# Load SAM
sam = LangSAM()

GroundingDINO_SwinB.cfg.py:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

final text_encoder_type: bert-base-uncased


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

groundingdino_swinb_cogcoor.pth:   0%|          | 0.00/938M [00:00<?, ?B/s]

Downloading: "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth" to /root/.cache/torch/hub/checkpoints/sam_vit_h_4b8939.pth
100%|██████████| 2.39G/2.39G [00:15<00:00, 168MB/s]


# Run Model

## Define Grid Search Parameters

In [5]:
from IPython.display import clear_output

## Tree

In [6]:
# # Define images to process
# imgs = [
#     "image_50001_2016-01-10", "image_50001_2016-03-06",
#     "image_5001_2016-06-05",
#     "image_73001_2018-03-04",
#     "image_50001_2017-07-23",
#     "image_54001_2016-02-07", "image_54001_2018-02-18",
#     "image_76001_2018-02-25",
#     "image_5001_2018-02-11"
# ]

# # Text and box thresholds
# tts = [0.1, 0.2, 0.3, 0.4, 0.5]
# bts = [0.1, 0.2, 0.3, 0.4, 0.5]

# # Types
# types = ["gamma", "rgb"]

# # Prompts
# tree_prompts = ['tree', 'vegetation', 'plants']

# # Counter
# total_iterations = len(imgs) * len(tts) * len(bts) * len(types) * len(tree_prompts)

# counter = 0
# for tt in tts:
#     for bt in bts:
#         for img in imgs:
#             for typ in types:
#                 for prompt in tree_prompts:
#                     train_individual_model(
#                         sam = sam,
#                         image = img,
#                         folder = typ,
#                         prompt = prompt,
#                         bt = bt,
#                         tt = tt,
#                         object_being_predicted = "Tree"
#                     )
#                 counter += 1
#                 if counter % 100 == 0:
#                     print(f'{counter}/total_iterations')

# Clouds

In [7]:
# # Define images to process
# imgs = [
#     "image_50001_2016-01-10", "image_50001_2016-03-06",
#     "image_5001_2016-06-05",
#     "image_73001_2018-03-04",
#     "image_50001_2017-07-23",
#     "image_54001_2016-02-07", "image_54001_2018-02-18",
#     "image_76001_2018-02-25",
#     "image_5001_2018-02-11"
# ]

# # Text and box thresholds
# tts = [0.1, 0.2, 0.3, 0.4, 0.5]
# bts = [0.1, 0.2, 0.3, 0.4, 0.5]

# # Types
# types = ["gamma", "rgb"]

# # Prompts
# cloud_prompts = ['cloud']

# # Counter
# total_iterations = len(imgs) * len(tts) * len(bts) * len(types) * len(cloud_prompts)

# counter = 0
# for tt in tts:
#     for bt in bts:
#         for img in imgs:
#             for typ in types:
#                 for prompt in cloud_prompts:
#                     train_individual_model(
#                         sam = sam,
#                         image = img,
#                         folder = typ,
#                         prompt = prompt,
#                         bt = bt,
#                         tt = tt,
#                         object_being_predicted = "Cloud"
#                     )
#                 counter += 1
#                 if counter % 100 == 0:
#                     print(f'{counter}/total_iterations')

# Urban

In [8]:
# Define images to process
imgs = [
    "image_50001_2016-01-10", "image_50001_2016-03-06",
    "image_5001_2016-06-05",
    "image_73001_2018-03-04",
    "image_50001_2017-07-23",
    "image_54001_2016-02-07", "image_54001_2018-02-18",
    "image_76001_2018-02-25",
    "image_5001_2018-02-11"
]

# Text and box thresholds
tts = [0.1, 0.2, 0.3, 0.4, 0.5]
bts = [0.1, 0.2, 0.3, 0.4, 0.5]

# Types
types = ["gamma", "rgb"]

# Prompts
urban_prompts = ['building', 'roof']

# Counter
total_iterations = len(imgs) * len(tts) * len(bts) * len(types) * len(urban_prompts)

counter = 0
for tt in tts:
    for bt in bts:
        for img in imgs:
            for typ in types:
                for prompt in urban_prompts:
                    train_individual_model(
                        sam = sam,
                        image = img,
                        folder = typ,
                        prompt = prompt,
                        bt = bt,
                        tt = tt,
                        object_being_predicted = "Urban"
                    )
                counter += 1
                if counter % 10 == 0:
                    print(f'{counter}/total_iterations')

10/total_iterations
20/total_iterations
30/total_iterations
40/total_iterations
50/total_iterations
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
60/total_iterations
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
70/total_iterations
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
80/total_iterations
No objects found in the image.
No 

# Water

In [9]:
# Define images to process
imgs = [
    "image_50001_2016-01-10", "image_50001_2016-03-06",
    "image_5001_2016-06-05",
    "image_73001_2018-03-04",
    "image_50001_2017-07-23",
    "image_54001_2016-02-07", "image_54001_2018-02-18",
    "image_76001_2018-02-25",
    "image_5001_2018-02-11"
]

# Text and box thresholds
tts = [0.1, 0.2, 0.3, 0.4, 0.5]
bts = [0.1, 0.2, 0.3, 0.4, 0.5]

# Types
types = ["gamma", "rgb"]

# Prompts
water_prompts = ['river', 'stream']

# Counter
total_iterations = len(imgs) * len(tts) * len(bts) * len(types) * len(water_prompts)

counter = 0
for tt in tts:
    for bt in bts:
        for img in imgs:
            for typ in types:
                for prompt in water_prompts:
                    train_individual_model(
                        sam = sam,
                        image = img,
                        folder = typ,
                        prompt = prompt,
                        bt = bt,
                        tt = tt,
                        object_being_predicted = "Water"
                    )
                counter += 1
                if counter % 10 == 0:
                    print(f'{counter}/total_iterations')
runtime.unassign()

10/total_iterations
20/total_iterations
30/total_iterations
40/total_iterations
50/total_iterations
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
60/total_iterations
No objects found in the image.
No objects found in the image.
No objects found in the image.
70/total_iterations
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
80/total_iterations
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
No objects found in the image.
90/total_iterations
100/total_iterations
110/total_iterations
120/total_iterations
130/total_ite